# Procesamiento de Datos de Dependencias Económicas
## Real Instituto Elcano - Índice de Seguridad Económica

Este notebook procesa los archivos CSV de dependencias económicas y los convierte a formato Parquet para análisis eficiente.

## 1. Instalación y Configuración

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import re
import gzip
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

## 2. Configuración de Rutas y Parámetros

In [2]:

# ============================================
# CONFIGURACIÓN - Ajusta estas rutas
# ============================================

base_path = Path.cwd().parent
INPUT_DIR = base_path / "data" / "processed" / "dependencias_consolidadas"

OUTPUT_DIR = base_path / "data" / "processed" / "parquet_files"
OUTPUT_FILE = OUTPUT_DIR / "dependencies_full.parquet"

# Configuración de redondeo
ROUNDING_CONFIG = {
    'dependency_value': 4,
    'direct_dependency': 4,
    'indirect_dependency': 4,
    'trade_value': 2,
    'longitud_optima': 0  # Convertir a entero
}

print(f"📂 Directorio de entrada: {INPUT_DIR.absolute()}")
print(f"📂 Directorio de salida: {OUTPUT_DIR.absolute()}")

📂 Directorio de entrada: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\dependencias_consolidadas
📂 Directorio de salida: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\parquet_files


## 3. Detección de Archivos

In [3]:
def find_dependency_files(directory: Path) -> List[tuple]:
    """
    Encuentra todos los archivos dependencias{año}.csv.gz
    Retorna lista de tuplas (año, path_completo)
    """
    pattern = re.compile(r'dependencias(\d{4})\.csv\.gz')
    files = []
    
    for file_path in directory.glob("dependencias*.csv.gz"):
        match = pattern.match(file_path.name)
        if match:
            year = int(match.group(1))
            files.append((year, file_path))
    
    return sorted(files, key=lambda x: x[0])

# Buscar archivos
files = find_dependency_files(INPUT_DIR)

print(f"✅ Archivos encontrados: {len(files)}\n")
for year, file_path in files:
    file_size = file_path.stat().st_size / 1024  # KB
    print(f"  📄 {file_path.name} - {file_size:,.1f} KB - Año {year}")

✅ Archivos encontrados: 22

  📄 dependencias2001.csv.gz - 40,891.1 KB - Año 2001
  📄 dependencias2002.csv.gz - 41,853.4 KB - Año 2002
  📄 dependencias2003.csv.gz - 42,215.3 KB - Año 2003
  📄 dependencias2004.csv.gz - 42,825.4 KB - Año 2004
  📄 dependencias2005.csv.gz - 43,570.1 KB - Año 2005
  📄 dependencias2006.csv.gz - 43,865.7 KB - Año 2006
  📄 dependencias2007.csv.gz - 44,631.4 KB - Año 2007
  📄 dependencias2008.csv.gz - 44,894.6 KB - Año 2008
  📄 dependencias2009.csv.gz - 44,911.0 KB - Año 2009
  📄 dependencias2010.csv.gz - 44,563.1 KB - Año 2010
  📄 dependencias2011.csv.gz - 44,995.7 KB - Año 2011
  📄 dependencias2012.csv.gz - 45,293.8 KB - Año 2012
  📄 dependencias2013.csv.gz - 45,774.8 KB - Año 2013
  📄 dependencias2014.csv.gz - 46,006.0 KB - Año 2014
  📄 dependencias2015.csv.gz - 46,006.0 KB - Año 2015
  📄 dependencias2016.csv.gz - 46,037.1 KB - Año 2016
  📄 dependencias2017.csv.gz - 46,385.9 KB - Año 2017
  📄 dependencias2018.csv.gz - 46,716.3 KB - Año 2018
  📄 dependencias20

## 4. Lectura y Procesamiento de Datos

In [4]:
def read_and_process_file(file_path: Path, year: int) -> pd.DataFrame:
    """
    Lee un archivo CSV.gz, procesa los datos y añade el año
    """
    print(f"⏳ Procesando {file_path.name}...")
    
    # Leer CSV comprimido
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        df = pd.read_csv(f, sep=';')
    
    # Añadir columna de año
    df['year'] = year
    
    # Redondear columnas numéricas
    for column, decimals in ROUNDING_CONFIG.items():
        if column in df.columns:
            if decimals == 0:
                df[column] = df[column].round(decimals).astype('Int64')  # Int64 permite NaN
            else:
                df[column] = df[column].round(decimals)
    
    print(f"  ✓ {len(df):,} registros cargados")
    
    return df

# Procesar cada archivo
dataframes = []
for year, file_path in files:
    df = read_and_process_file(file_path, year)
    dataframes.append(df)

print("\n✅ Todos los archivos procesados")

⏳ Procesando dependencias2001.csv.gz...
  ✓ 6,795,998 registros cargados
⏳ Procesando dependencias2002.csv.gz...
  ✓ 6,921,142 registros cargados
⏳ Procesando dependencias2003.csv.gz...
  ✓ 6,980,132 registros cargados
⏳ Procesando dependencias2004.csv.gz...
  ✓ 7,053,498 registros cargados
⏳ Procesando dependencias2005.csv.gz...
  ✓ 7,158,420 registros cargados
⏳ Procesando dependencias2006.csv.gz...
  ✓ 7,241,250 registros cargados
⏳ Procesando dependencias2007.csv.gz...
  ✓ 7,293,002 registros cargados
⏳ Procesando dependencias2008.csv.gz...
  ✓ 7,293,870 registros cargados
⏳ Procesando dependencias2009.csv.gz...
  ✓ 7,297,698 registros cargados
⏳ Procesando dependencias2010.csv.gz...
  ✓ 7,277,098 registros cargados
⏳ Procesando dependencias2011.csv.gz...
  ✓ 7,380,706 registros cargados
⏳ Procesando dependencias2012.csv.gz...
  ✓ 7,408,278 registros cargados
⏳ Procesando dependencias2013.csv.gz...
  ✓ 7,470,004 registros cargados
⏳ Procesando dependencias2014.csv.gz...
  ✓ 7,470,1

## 5. Consolidación de Datos y compactación en memoria

In [5]:
# Consolidar todos los dataframes
print("🔄 Consolidando datos...")
df_full = pd.concat(dataframes, ignore_index=True)
print(f"✅ Dataset consolidado con {len(df_full):,} registros")

# %%
# --- OPTIMIZACIÓN DE MEMORIA ---

print("📊 ESTADO DE MEMORIA (ANTES):")
# deep=True es crucial para medir el tamaño real de las columnas 'object'
memoria_antes = df_full.memory_usage(deep=True).sum()
print(f"{memoria_antes / (1024**3):.2f} GiB") # Memoria en Gigabytes

# 1. Convertir 'object' a 'category'
df_full['industry'] = df_full['industry'].astype('category')
df_full['dependent_country'] = df_full['dependent_country'].astype('category')
df_full['supplier_country'] = df_full['supplier_country'].astype('category')

# 2. Convertir 'float64' a 'float32'
df_full['dependency_value'] = df_full['dependency_value'].astype('float32')
df_full['direct_dependency'] = df_full['direct_dependency'].astype('float32')
df_full['indirect_dependency'] = df_full['indirect_dependency'].astype('float32')
df_full['trade_value'] = df_full['trade_value'].astype('float32')

# 3. Convertir 'int64' a 'int16'
df_full['year'] = df_full['year'].astype('int16')

print("\n📊 ESTADO DE MEMORIA (DESPUÉS):")
memoria_despues = df_full.memory_usage(deep=True).sum()
print(f"{memoria_despues / (1024**3):.2f} GiB")

print("\n📈 NUEVOS TIPOS DE DATOS:")
print(df_full.dtypes)

print(f"\n✅ Memoria reducida en un {((memoria_antes - memoria_despues) / memoria_antes) * 100:.1f}%")



🔄 Consolidando datos...
✅ Dataset consolidado con 159,672,930 registros
📊 ESTADO DE MEMORIA (ANTES):
34.29 GiB

📊 ESTADO DE MEMORIA (DESPUÉS):
4.91 GiB

📈 NUEVOS TIPOS DE DATOS:
industry               category
dependent_country      category
supplier_country       category
dependency_value        float32
direct_dependency       float32
indirect_dependency     float32
trade_value             float32
longitud_optima           Int64
year                      int16
dtype: object

✅ Memoria reducida en un 85.7%


## 6. Exploración del Dataset

In [6]:
# Información general
print("📊 INFORMACIÓN DEL DATASET\n")
print(f"Total de registros: {len(df_full):,}")
print(f"Columnas: {list(df_full.columns)}")
print(f"\nTipos de datos:")
print(df_full.dtypes)

# %%
# Estadísticas por año
print("\n📈 DISTRIBUCIÓN POR AÑO\n")
year_stats = df_full.groupby('year').size()
print(year_stats)

# %%
# Estadísticas descriptivas
print("\n📊 ESTADÍSTICAS DESCRIPTIVAS\n")
print(df_full.describe())

# %%
# Información de dimensiones
print("\n🌍 DIMENSIONES DEL DATASET\n")
print(f"Años: {sorted(df_full['year'].unique())}")
print(f"Países dependientes únicos: {df_full['dependent_country'].nunique()}")
print(f"Países proveedores únicos: {df_full['supplier_country'].nunique()}")
print(f"Industrias únicas: {df_full['industry'].nunique()}")

# %%
# Muestra de datos
print("\n👀 PRIMERAS FILAS DEL DATASET\n")
df_full.head(10)

📊 INFORMACIÓN DEL DATASET

Total de registros: 159,672,930
Columnas: ['industry', 'dependent_country', 'supplier_country', 'dependency_value', 'direct_dependency', 'indirect_dependency', 'trade_value', 'longitud_optima', 'year']

Tipos de datos:
industry               category
dependent_country      category
supplier_country       category
dependency_value        float32
direct_dependency       float32
indirect_dependency     float32
trade_value             float32
longitud_optima           Int64
year                      int16
dtype: object

📈 DISTRIBUCIÓN POR AÑO

year
2001    6795998
2002    6921142
2003    6980132
2004    7053498
2005    7158420
2006    7241250
2007    7293002
2008    7293870
2009    7297698
2010    7277098
2011    7380706
2012    7408278
2013    7470004
2014    7470140
2015    7470140
2016    7316528
2017    7354630
2018    7365314
2019    7344572
2020    7279508
2021    7302038
2022    7198964
dtype: int64

📊 ESTADÍSTICAS DESCRIPTIVAS

       dependency_value  di

,industry,dependent_country,supplier_country,dependency_value,direct_dependency,indirect_dependency,trade_value,longitud_optima,year
0,156 Transport,ARE,AZE,1.0,1.0,0.0,12.88,2,2001
1,156 Transport,BMU,USA,1.0,1.0,0.0,123.00,2,2001
2,156 Transport,BRN,AUS,1.0,1.0,0.0,9.84,2,2001
3,156 Transport,DZA,TUN,1.0,1.0,0.0,7.23,2,2001
4,156 Transport,FJI,AUS,1.0,1.0,0.0,15.01,2,2001
5,156 Transport,GEO,AZE,1.0,1.0,0.0,7.65,2,2001
6,156 Transport,KAZ,AZE,1.0,1.0,0.0,5.51,2,2001
7,156 Transport,KGZ,AZE,1.0,1.0,0.0,0.04,2,2001
8,156 Transport,LBY,TUN,1.0,1.0,0.0,10.85,2,2001
9,156 Transport,MDA,AZE,1.0,1.0,0.0,0.01,2,2001


## 7. Verificación de Redondeo

In [7]:

# Verificar que el redondeo funcionó correctamente
print("🔍 VERIFICACIÓN DE DECIMALES\n")
for col in ['dependency_value', 'direct_dependency', 'indirect_dependency']:
    if col in df_full.columns:
        sample = df_full[col].dropna().head(5)
        print(f"{col}:")
        for val in sample:
            print(f"  {val}")
        print()

# %% [markdown]

🔍 VERIFICACIÓN DE DECIMALES

dependency_value:
  1.0
  1.0
  1.0
  1.0
  1.0

direct_dependency:
  1.0
  1.0
  1.0
  1.0
  1.0

indirect_dependency:
  0.0
  0.0
  0.0
  0.0
  0.0



## 8. Guardado en Formato Parquet

In [8]:
OUTPUT_FILE

WindowsPath('c:/Users/Usuario/Documents/Github/Seguridad Economica/data/processed/parquet_files/dependencies_full.parquet')

In [9]:

# Crear directorio de salida
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Guardar a Parquet
print(f"💾 Guardando archivo Parquet en: {OUTPUT_FILE}\n")

df_full.to_parquet(
    OUTPUT_FILE,
    engine='pyarrow',
    compression='snappy',
    index=False
)

# Calcular tamaños
parquet_size = OUTPUT_FILE.stat().st_size / 1024**2  # MB
original_size = sum(f.stat().st_size for _, f in files) / 1024**2  # MB
compression_ratio = (1 - parquet_size/original_size) * 100

print(f"✅ Archivo generado exitosamente")
print(f"\n📊 COMPARACIÓN DE TAMAÑOS:")
print(f"  CSV.gz originales: {original_size:.2f} MB")
print(f"  Parquet generado: {parquet_size:.2f} MB")
print(f"  Compresión adicional: {compression_ratio:.1f}%")

💾 Guardando archivo Parquet en: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\parquet_files\dependencies_full.parquet

✅ Archivo generado exitosamente

📊 COMPARACIÓN DE TAMAÑOS:
  CSV.gz originales: 966.55 MB
  Parquet generado: 168.85 MB
  Compresión adicional: 82.5%


## 9. Verificación de Lectura

In [10]:
# Verificar que el archivo Parquet se puede leer correctamente
print("🔍 Verificando archivo Parquet...\n")

df_test = pd.read_parquet(OUTPUT_FILE)
print(f"✅ Archivo leído correctamente")
print(f"Registros: {len(df_test):,}")
print(f"Columnas: {list(df_test.columns)}")
print(f"\nPrimeras filas:")
df_test.head()

🔍 Verificando archivo Parquet...

✅ Archivo leído correctamente
Registros: 159,672,930
Columnas: ['industry', 'dependent_country', 'supplier_country', 'dependency_value', 'direct_dependency', 'indirect_dependency', 'trade_value', 'longitud_optima', 'year']

Primeras filas:


,industry,dependent_country,supplier_country,dependency_value,direct_dependency,indirect_dependency,trade_value,longitud_optima,year
0,156 Transport,ARE,AZE,1.0,1.0,0.0,12.88,2,2001
1,156 Transport,BMU,USA,1.0,1.0,0.0,123.00,2,2001
2,156 Transport,BRN,AUS,1.0,1.0,0.0,9.84,2,2001
3,156 Transport,DZA,TUN,1.0,1.0,0.0,7.23,2,2001
4,156 Transport,FJI,AUS,1.0,1.0,0.0,15.01,2,2001


## 10. Resumen Final

In [11]:

print("="*60)
print("✅ PROCESAMIENTO COMPLETADO")
print("="*60)
print(f"\n📁 Archivo generado: {OUTPUT_FILE}")
print(f"📊 Total de registros: {len(df_full):,}")
print(f"📅 Años incluidos: {sorted(df_full['year'].unique())}")
print(f"💾 Tamaño: {parquet_size:.2f} MB")
print("\n🎯 Siguiente paso: Análisis y visualización de datos")

✅ PROCESAMIENTO COMPLETADO

📁 Archivo generado: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\parquet_files\dependencies_full.parquet
📊 Total de registros: 159,672,930
📅 Años incluidos: [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
💾 Tamaño: 168.85 MB

🎯 Siguiente paso: Análisis y visualización de datos
